# Explore Dataset

In [2]:
import transformers
import sentencepiece as spm

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv(r"C:\Users\Admin\Downloads\finetunet5\lang8dataset.csv") #lấy dự liệu đã được tiền xử lý từ code encoder - decoder
df.head()

,correct,error
0,And he took in my favorite subjects like soccer .,And he took in my favorite subject like soccer .
1,"Actually , he was the one who let me know abou...","Actually , who let me know about Lang - was him ."
2,His Kanji ability is much better than mine .,His Kanji is ability is much better than me .
3,I heard a sentence last night when I was watch...,I heard a sentence last night when I watched TV .
4,"When you go uphill , you have to bend your back .","When you go uphill , you hvae to bend your back ."


In [10]:
df.shape #dữ liệu gồm 289562 dòng và 2 cột: Correct và error

(289562, 2)

In [18]:
df.rename(columns={'correct': 'output', 'error': 'input'}, inplace=True)# đổi tên cột correct thành output và error thành input để đưa vào model


In [19]:
df.head() # in ra 5 dòng đầu tiên của dữ liệu sau khi đã đổi tên cột

,output,input
0,And he took in my favorite subjects like soccer .,And he took in my favorite subject like soccer .
1,"Actually , he was the one who let me know abou...","Actually , who let me know about Lang - was him ."
2,His Kanji ability is much better than mine .,His Kanji is ability is much better than me .
3,I heard a sentence last night when I was watch...,I heard a sentence last night when I watched TV .
4,"When you go uphill , you have to bend your back .","When you go uphill , you hvae to bend your back ."


In [11]:
dataset_size = df.shape #kích thước của dữ liệu. Bao gồm 289562
missing_values = df.isnull().sum() #kiểm tra số lượng giá trị null trong từng cột (output, input)

dataset_size, missing_values

((289562, 2),
 correct    0
 error      0
 dtype: int64)

# Train

## Importing libraries

In [12]:
import pandas as pd
from tqdm import tqdm # thư viện hỗ trợ hiển thị tiến trình của vòng lặp
import argparse # thư viện hỗ trợ phân tích tham số đầu vào của chương trình
import glob # thư viện hỗ trợ tìm kiếm các tệp tin trong thư mục
import os #thao tác với các thư mục
import json #thư viện hỗ trợ làm việc với json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk #thư viện giúp hỗ trợ các tách văn bản thành câu, từ, ký tự
nltk.download('punkt') 
from nltk.tokenize import sent_tokenize #sent_tokenize: tách văn bản thành câu

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Set seed

In [13]:
def set_seed(seed): #hàm để thiết lập seed cho các thư viện random, numpy và torch
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)

set_seed(42)

## The dataset

In [20]:
#T5ForConditionalGeneration: giúp load mô hình T5 đã được huấn luyện trước đó
#T5Tokenizer: giúp mã hóa và giải mã câu đầu vào và đầu ra
#tokenizer: nhận vào câu đầu vào => chuyển thành các token, từ token chuyển thành id
from transformers import (
    T5ForConditionalGeneration, T5Tokenizer,
    Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
  )

from torch.utils.data import Dataset, DataLoader

In [21]:
model_name = 't5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name) #load tokenizer từ mô hình T5
model = T5ForConditionalGeneration.from_pretrained(model_name) #load mô hình T5 đã được huấn luyện trước đó
model.resize_token_embeddings(len(tokenizer))
print("Tokenizer vocab size:", len(tokenizer))
print("Model vocab size:", model.config.vocab_size)

Tokenizer vocab size: 32100
Model vocab size: 32100


In [22]:
input_text = ["He loves football", "I usually eat ice-cream in the summer","Watching TV is my favorite hobby"]
for  text in input_text:
    print(f"Input text: {text}")
    tokenized_text = tokenizer.tokenize(text)
    print(f"Tokenized text: {tokenized_text}")
    print("="*50)

Input text: He loves football
Tokenized text: ['▁He', '▁loves', '▁football']
Input text: I usually eat ice-cream in the summer
Tokenized text: ['▁I', '▁usually', '▁', 'eat', '▁', 'ice', '-', 'cream', '▁in', '▁the', '▁summer']
Input text: Watching TV is my favorite hobby
Tokenized text: ['▁Watch', 'ing', '▁TV', '▁is', '▁my', '▁favorite', '▁hobby']


In [23]:
#sử dụng tokenizer cho câu đầu tiên
input_text = df['input'][0] #lấy câu đầu vào đầu tiên từ dữ liệu
print("Input text:", input_text) #in ra câu đầu vào
print("Tokenized input:", tokenizer.tokenize(input_text)) #in ra các token của câu đầu vào
print("Tokenized input IDs:", tokenizer.encode(input_text)) #in ra các id của các token trong câu đầu vào
print(tokenizer(input_text, return_tensors="pt")) #in ra các tensor của câu đầu vào

Input text: And he took in my favorite subject like soccer .
Tokenized input: ['▁And', '▁', 'he', '▁took', '▁in', '▁my', '▁favorite', '▁subject', '▁like', '▁soccer', '▁', '.']
Tokenized input IDs: [275, 3, 88, 808, 16, 82, 1305, 1426, 114, 10749, 3, 5, 1]
{'input_ids': tensor([[  275,     3,    88,   808,    16,    82,  1305,  1426,   114, 10749,
             3,     5,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [24]:
def calc_token_len(example): #hàm tính độ dài của câu đầu vào và đầu ra sau khi tokenized
    return len(tokenizer(example).input_ids)

In [25]:
from sklearn.model_selection import train_test_split #sử dụng train_test_split để chia dữ liệu thành 2 phần: train và test
train_df, test_df = train_test_split(df, test_size=0.10, shuffle=True) # chia dữ liệu thành 2 phần: train và test với tỷ lệ 90% cho train và 10% cho test
train_df.shape, test_df.shape #in ra kích thước của dữ liệu train và test

((260605, 2), (28957, 2))

In [26]:
test_df['input_token_len'] = test_df['input'].apply(calc_token_len) #tính độ dài của câu đầu vào sau khi tokenized

test_df.head() #in ra độ dài của  5 câu đầu vào đầu tiên sau khi tokenized

,output,input,input_token_len
184679,I am sure The sunscreen attacked my eyes .,"The sunscreen attacked my eyes , I am sure .",13
57559,"I wrote , I am worried about them in my previo...",I wrote I am worried about them before this jo...,13
123457,I became a third year student .,I became a thirs year student .,13
165509,To become rich .,To become the rich .,7
88727,But I can not operate my tongue freely .,But I can not opperate my tongue freely .,14


In [27]:
print(test_df['input_token_len'].max()) #in ra độ dài lớn nhất của câu đầu vào sau khi tokenized

38


In [163]:
test_df['input_token_len'].describe() #in ra thống kê về độ dài của câu đầu vào sau khi tokenized

count    28957.000000
mean        13.282281
std          3.347163
min          1.000000
25%         11.000000
50%         13.000000
75%         16.000000
max         38.000000
Name: input_token_len, dtype: float64

In [28]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df) #tạo dataset dành cho train từ dataframe train_df
test_dataset = Dataset.from_pandas(test_df) #tạo dataset dành cho test từ dataframe test_df

In [29]:
test_dataset #in ra các thuộc tính của test_dataset
print(test_dataset[0]) #in ra câu đầu vào và đầu ra của test_dataset

{'output': 'I am sure The sunscreen attacked my eyes .', 'input': 'The sunscreen attacked my eyes , I am sure .', 'input_token_len': 13, '__index_level_0__': 184679}


In [30]:
from torch.utils.data import Dataset, DataLoader 
class GrammarDataset(Dataset): #lớp GrammarDataset: 
    def __init__(self, dataset, tokenizer,print_text=False): #nhận đầu vào là dataset, tokenizer
        self.dataset = dataset #dữ liệu đầu vào
        self.pad_to_max_length = False #không sử dụng padding thành max_length cho dữ liệu đầu vào
        self.tokenizer = tokenizer #nhận tokenizer
        self.print_text = print_text
        self.max_len = 64 #độ dài tối đa của câu đầu vào và đầu ra sau khi tokenized

    def __len__(self):
        return len(self.dataset) #trả về số câu của dataset


    def tokenize_data(self, example):
        input_, target_ = example['input'], example['output'] #input_ là câu đầu vào, target_ là câu đầu ra

        #tokenize câu đầu vào. paddding = False: không sử dụng padding cho câu đầu vào
        #max_length = self.max_len: nếu sau khi tokenized mà độ dài của câu đầu vào lớn hơn max_len thì sẽ cắt bớt đi để còn thành max_len
        #return_attention_mask=True: trả về attention mask cho câu đầu vào => giúp mô hình biết được đâu là padding và đâu là câu thật sự

        tokenized_inputs = tokenizer(input_, padding = False,
                                            max_length=self.max_len,
                                            truncation=True,
                                            return_attention_mask=True)

        tokenized_targets = tokenizer(target_, padding = False,
                                            max_length=self.max_len,
                                            truncation=True,
                                            return_attention_mask=True)

        inputs={"input_ids": tokenized_inputs['input_ids'],
            "attention_mask": tokenized_inputs['attention_mask'],
            "labels": tokenized_targets['input_ids']
        }
        #labels: là câu đầu ra sau khi tokenized
        #input_ids: là câu đầu vào sau khi tokenized
        #attention_mask: là mặt nạ attention cho câu đầu vào sau khi tokenized

        return inputs


    def __getitem__(self, index):
        inputs = self.tokenize_data(self.dataset[index]) #lấy câu đầu vào và đầu ra thứu i

        if self.print_text:
            for k in inputs.keys():
                print(k, len(inputs[k])) #in ra độ dài của các thuộc tính trong inputs

        return inputs

In [ ]:
dataset = GrammarDataset(test_dataset, tokenizer, False)
for i in [120]:
    print(f"Input id: {dataset[i]['input_ids']}")
    print(f"Attention mask: {dataset[i]['attention_mask']}")
    print(f"Labels: {dataset[i]['labels']}")

Input id: [1615, 410, 59, 27, 369, 1461, 3, 5, 1]
Attention mask: [1, 1, 1, 1, 1, 1, 1, 1, 1]
Labels: [1615, 410, 59, 27, 281, 269, 550, 1]


: 

## Evaluator

In [175]:
import evaluate #thư viện tạo các ma trận đánh giá cho mô hình
rouge_metric = evaluate.load("rouge")

## Train Model

In [180]:
#áp dụng data_collator cho dữ liệu đầu vào
#tokenizer: là tokenizer đã được load từ mô hình T5
#model: là mô hình T5 
#padding='longest': sử dụng padding cho câu đầu vào và đầu ra thành độ dài lớn nhất trong batch (batch_size lấy từ Trainer)
#return_tensors='pt': trả về tensor cho câu đầu vào và đầu ra
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding='longest', return_tensors='pt') 

In [181]:
print(data_collator([dataset[100]])) #in ra dữ liệu đầu vào và đầu ra sau khi áp dụng data_collator
print(dataset[100])

input_ids 16
attention_mask 16
labels 16
{'input_ids': tensor([[   27,  2944,    46, 15417,   945,  3270,     3,     6,   250,     8,
           774,   141,   483,     3,     5,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[   27,  2944,    46, 15417, 11999,  3270,     3,     6,   250,     8,
           774,   141,  2130,     3,     5,     1]]), 'decoder_input_ids': tensor([[    0,    27,  2944,    46, 15417, 11999,  3270,     3,     6,   250,
             8,   774,   141,  2130,     3,     5]])}
input_ids 16
attention_mask 16
labels 16
{'input_ids': [27, 2944, 46, 15417, 945, 3270, 3, 6, 250, 8, 774, 141, 483, 3, 5, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [27, 2944, 46, 15417, 11999, 3270, 3, 6, 250, 8, 774, 141, 2130, 3, 5, 1]}


In [ ]:
batch_size = 16
#seq2seq_trainer: là lớp huấn luyện mô hình T5
#args: là các tham số đầu vào cho mô hình T5
args = Seq2SeqTrainingArguments(output_dir=r"C:\Users\Admin\Downloads\finetunet5", #thư mục lưu trữ mô hình đã được huấn luyện
                        eval_strategy="steps", #đánh giá mô hình sau mỗi batch
                        per_device_train_batch_size=batch_size, #batch_size cho train
                        per_device_eval_batch_size=batch_size, #batch_size cho eval
                        learning_rate=2e-5, #tốc độ học của mô hình
                        num_train_epochs=5, #số lượng epoch cho mô hình
                        weight_decay=0.01, #thêm vào hàm loss là weigth_decay* độ lớn của trọng số => giảm overfitting
                        save_total_limit=2, #giới hạn số lượng mô hình đã được lưu trữ
                        predict_with_generate=True, #sử dụng hàm generate để dự đoán đầu ra cho câu đầu vào
                        fp16 = True, #huấn luyệ với fp16 (sử dụng 16 bit để huấn luyện mô hình)
                        gradient_accumulation_steps = 6,
                        eval_steps = 250, #đánh giá mô hình sau mỗi 250 bước
                        save_steps = 250, #lưu trữ mô hình sau mỗi 250 bước
                        load_best_model_at_end=True, #sau khi train xong, tự động lưu lại checkpoint tốt nhất
                        logging_dir="/logs", #thư mục lưu trữ log
                        report_to="wandb") #sử dụng wandb để theo dõi quá trình huấn luyện mô hình

In [184]:
import nltk
nltk.download('punkt')
from nltk.translate.gleu_score import sentence_gleu
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = [nltk.sent_tokenize(pred.strip()) for pred in decoded_preds]
    decoded_labels = [nltk.sent_tokenize(label.strip()) for label in decoded_labels]

    gleu_scores = [sentence_gleu([ref], pred) for pred, ref in zip(decoded_preds, decoded_labels)]
    result = {"gleu": np.mean(gleu_scores) * 100}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [185]:
print("Tokenizer vocab size:", len(tokenizer))
print("Model vocab size:", model.config.vocab_size)


Tokenizer vocab size: 32100
Model vocab size: 32100


In [186]:
#bắt đầu train model
trainer = Seq2SeqTrainer(model=model, #truyền vào model cần train
                args=args, #truyền vào các tham số khi training cho model
                train_dataset= GrammarDataset(train_dataset, tokenizer), #truyền vào dữ liệu train đã được xử lý
                eval_dataset=GrammarDataset(test_dataset, tokenizer), #truyền vào dữ liệu test đã được xử lý
                processing_class = tokenizer, #truyền vào tokenizer đã được load từ mô hình T5
                data_collator=data_collator, #truyền vào data_collator đã được tạo ở trên
                compute_metrics=compute_metrics) #truyền vào hàm compute_metrics để đánh giá mô hình


In [187]:
print("Tokenizer vocab size:", len(tokenizer))
print("Model vocab size:", model.config.vocab_size)


Tokenizer vocab size: 32100
Model vocab size: 32100


In [188]:
#in ra token đặc biệt trong tokenizer
print(tokenizer.special_tokens_map) #in ra các token đặc biệt trong tokenizer

{'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_id_44>', '<extra_id_45>', '<extra_id_46>', '<extra_id_47>', '<extra_id_48>', '<extra_id_49>', '<extra_id_50>', '<extra_id_51>', '<extra_id_52>', '<extra_id_53

In [189]:
trainer.train()

Step,Training Loss,Validation Loss,Gleu,Gen Len
250,No log,0.911609,0.921700,4.379600
500,1.067900,0.861993,1.371800,4.575900
750,1.067900,0.837362,1.592900,4.614600
1000,0.941400,0.820891,1.679800,4.518900
1250,0.941400,0.809451,1.686800,4.415700
1500,0.912000,0.796989,1.758700,4.316200
1750,0.912000,0.790284,1.822000,4.298000
2000,0.882400,0.784275,1.773100,4.142600
2250,0.882400,0.779560,1.819100,4.136200
2500,0.868800,0.774255,1.816800,4.020400


KeyboardInterrupt: 

In [ ]:
trainer.save_model('model')